Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [4]:
# Read a few reviews from the "Excellent" category

pd.set_option("display.max_colwidth", 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
3655,4999,"\nThey blended Wild Weasel single malt with whisky from the Radermacher distillery (maker of Lambertus grain whisky) to create the world’s first blended Belgian whisky. It noses young, with straw, digestive biscuits, damp woodlands, oat breakfast cereals, and a hint of spice saving it from nosing like a crisp Sauvignon Blanc. The mouthfeel is languorous, with a musty texture, some grapefruit acidity, tangerine sweetness, and a mild, malty finish. So now we know. €40",0
482,3943,"\nThis summons up plenty of snappy grain notes, with green apple, melon rind, and zested lemon bumping into dried chili, green tomato, damp hay bales, and a dose of spice. It’s sweeter than a kitten with a ball of wool: lemon bonbons, toffee, vanilla cream, banana chews, zesty mandarin, apple, butterscotch, pepper, and clove. Sour apple pips, burnt sugar and spice, and some residual vanilla creaminess see it out.",0
1744,4788,"\nThis is the Lowland entry into Douglas Laing’s Remarkable Regional Malts collection, and on approach the nose is like sipping lemonade in a malt hopper: gristy malt, lemon, lime, honeydew melon, vanilla pods, peach, and apricot. A similarly juicy palate, gaining sweet grassy notes, Spangles, and confectioners’ sugar, before ending on a finish of dried citrus and fizzy Refreshers. True to the region, but the Epicurean makes the rest of the range look brighter, quicker, smarter, and tastier in comparison. £34",0


In [5]:
# Read a few reviews from the "Poor" category

pd.set_option("display.max_colwidth", 0)
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
2437,5058,"\nOne of the Barton whiskies acquired by Sazerac in 2009, Canadian Supreme is a dry, dusty mixer that blossoms in ginger ale. Neat, the nose opens slowly to dried fruit, while the palate is grassy, almost mashy, with butterscotch pudding and a peppery glow. Though not at all tannic, still it becomes a bit pulling in the middle before giving way to prunes, raspberries, and maraschino cherries. The finish is surprisingly long and hot.",2
3508,5099,"\nReal maple syrup has an earthy, woodsy aroma; maple flavoring has strong overtones of coconut, and so does Cabin Fever. The nose evokes dried, sweetened baking coconut, while the sweet and spicy palate is a hot, liquid, coconut macaroon. Peppery notes suggest that it’s whisky, but without any traces of barrel aging that’s as close as you get. Best part? The long, spicy finish with its confection-sweet coconut.",2
2396,5088,"\nOne of the most popular flavored whiskies today is Fireball cinnamon, so it’s no surprise that other cinnamon whiskies are entering the market. This one has a lovely, woody, cinnamon nose that bursts into sweet, blistering cinnamon on the palate. Cinnamon is a natural whisky flavor, but here, rather than complement the underlying whisky, it completely masks it. This is a cinnamon liqueur, and a good one. A fun shooter, perhaps, but it’s barely whisky.",2


# Split the Training Set into Train/Validation

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train["description"],
                                                   train["ratingCategory"], 
                                                   test_size=0.2, 
                                                   stratify=train["ratingCategory"], 
                                                   random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), 
                 ('rfc', rfc)])

# **SUBMISSION 1**

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__max_features': randint(5000, 10000),
    'rfc__n_estimators': randint(100, 1000),
    'rfc__max_depth':randint(1,20),
    'rfc__max_leaf_nodes': randint(1, 10000),
    'rfc__min_samples_split': randint(2, 40),
    'rfc__min_samples_leaf': randint(1, 20),
}

random_search = RandomizedSearchCV(pipe, parameters, cv=30, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

Fitting 30 folds for each of 10 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.0min finished


RandomizedSearchCV(cv=30, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                       

In [9]:
from sklearn.metrics import accuracy_score
pred = random_search.predict(X_test)

accuracy_score(y_test, pred)

0.7102689486552567

In [10]:
random_search.best_score_

0.7121474685694869

In [11]:
random_search.best_params_

{'rfc__max_depth': 14,
 'rfc__max_leaf_nodes': 8055,
 'rfc__min_samples_leaf': 1,
 'rfc__min_samples_split': 29,
 'rfc__n_estimators': 203,
 'vect__max_df': 0.75,
 'vect__max_features': 5221}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [12]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [13]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [14]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [15]:
subNumber = 1

In [16]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

# **SUBMISSION 2**

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [17]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

sgdc = SGDClassifier()

lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', sgdc)])
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [18]:
parameters = {
    'lsi__svd__n_components': [10, 100, 150, 200, 250, 300, 400],
}

grid_search = GridSearchCV(pipe, parameters, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed: 17.5min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
            

In [19]:
from sklearn.metrics import accuracy_score
pred = grid_search.predict(X_test)

accuracy_score(y_test, pred)

0.7444987775061125

In [20]:
grid_search.best_score_

0.7497748635110035

In [21]:
grid_search.best_params_

{'lsi__svd__n_components': 400}

### Make a Submission File

In [22]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [23]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [24]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [25]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# **SUBMISSION 3**

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [26]:
# Continue Word Embedding Work Here
import spacy

nlp = spacy.load("en_core_web_lg")

In [27]:
def get_words_vectors(docs):
    return[nlp(doc).vector for doc in docs]

In [28]:
X = get_words_vectors(X_train)

In [29]:
#Define Pipeline
pipe = Pipeline([
    ('clf', rfc)
])

In [30]:
parameters = {
    'clf__n_estimators': (25, 1500),
    'clf__max_depth': (15, 30)
}

grid_search = GridSearchCV(pipe, parameters, cv=6, n_jobs=-1, verbose=1)
grid_search.fit(X, y_train)

Fitting 6 folds for each of 4 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  4.3min finished


GridSearchCV(cv=6, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clf',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               max_samples=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                              

In [31]:
X_test_spacy = get_words_vectors(X_test)

In [32]:
from sklearn.metrics import accuracy_score
pred = grid_search.predict(X_test_spacy)

accuracy_score(y_test, pred)

0.7408312958435208

In [33]:
grid_search.best_score_

0.7363121739521498

In [34]:
grid_search.best_params_

{'clf__max_depth': 30, 'clf__n_estimators': 1500}

### Make a Submission File

In [40]:
# Predictions on test sample
test_1 = get_words_vectors(test["description"])
pred = grid_search.predict(test_1)

In [41]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [42]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [43]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

1. What is Sentiment Analysis?
- computationally identifying and categorizing opinions in a piece of text. 
- The main difference between classification and sentiment analysis is the idea that classification is objective but sentiment is subjective.
- Some examples:
    Reputation Management - Social Media Monitoring - Brand Monitoring.
    Market Research, Competitor Analysis.
    Product Analytics.
    Customer Analysis.
    Customer Support.

2. Why did word embedding worked better for the lecture notebook than on the whiskey competition?
- This dataset deals with sentiment analysis rather than true document classification. There are a lot of subjective words that are more difficult to classify utilizing the given methods.
- Convolutional neural networks are effective at document classification, namely because they are able to pick out salient features (e.g. tokens or sequences of tokens) in a way that is invariant to their position within the input sequences.